In [ ]:
# ================================
#  Environment setup (Colab)
# ================================
!pip -q install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
!pip -q install torch-geometric torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.3.0+cu121.html
print("✅ PyTorch & PyG install attempted.")

✅ PyTorch & PyG install attempted.


In [ ]:
# ================================
#  Imports & Utilities
# ================================
import os, random, copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GCNConv, SAGEConv, global_mean_pool, global_add_pool
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_batch
from sklearn.metrics import mean_squared_error

# Device & seed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

Device: cpu


In [ ]:
# ================================
#  Load AIDS dataset for graph matching
# ================================
# Load AIDS dataset
dataset = TUDataset(root='/content/data/AIDS', name='AIDS')
print(f"Dataset: {dataset}")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
print(f"Number of classes: {dataset.num_classes}")

# Create graph pairs for matching task
def create_graph_pairs(dataset, num_pairs=1000, seed=42):
    """Create pairs of graphs with similarity scores for matching task"""
    set_seed(seed)
    pairs = []
    similarities = []

    for _ in range(num_pairs):
        # Randomly select two graphs
        idx1, idx2 = torch.randint(0, len(dataset), (2,)).tolist()
        graph1 = dataset[idx1]
        graph2 = dataset[idx2]

        # Simple similarity based on class labels and graph size
        if graph1.y.item() == graph2.y.item():
            # Same class: higher similarity
            base_sim = 0.7 + 0.3 * torch.rand(1).item()
        else:
            # Different class: lower similarity
            base_sim = 0.1 + 0.4 * torch.rand(1).item()

        # Adjust based on size difference
        size_diff = abs(graph1.num_nodes - graph2.num_nodes) / max(graph1.num_nodes, graph2.num_nodes)
        adjusted_sim = base_sim * (1 - 0.3 * size_diff)

        pairs.append((graph1, graph2))
        similarities.append(adjusted_sim)

    return pairs, similarities

# Create training, validation, and test pairs
train_pairs, train_sims = create_graph_pairs(dataset, num_pairs=800, seed=1)
val_pairs, val_sims = create_graph_pairs(dataset, num_pairs=100, seed=2)
test_pairs, test_sims = create_graph_pairs(dataset, num_pairs=100, seed=3)

num_feats = dataset.num_features
print(f"Features: {num_feats}")
print(f"Train/Val/Test pairs: {len(train_pairs)}/{len(val_pairs)}/{len(test_pairs)}")


Dataset: AIDS(2000)
Number of graphs: 2000
Number of features: 38
Number of classes: 2
Features: 38
Train/Val/Test pairs: 800/100/100


In [ ]:
# ================================
#  Config for AIDS graph matching
# ================================
CFG = dict(
    POS_TRAIN=15, POS_TEST=15,    # Reduced from 30
    NEG_TRAIN=15, NEG_TEST=15,    # Reduced from 30
    USE_FT_LAST=True,
    USE_FT_ALL=False,
    USE_PR_LAST=False,
    USE_PR_ALL=False,
    USE_DISTILL=False,            # Disable first
    DISTILL_STEPS=50,
    FP_P=32,                      # Reduced from 64
    FP_NODES=16,
    FP_EDGE_INIT_P=0.1,
    FP_EDGE_TOPK=24,
    EDGE_LOGIT_STEP=2.0,
    OUTER_ITERS=15,               # Slightly increased
    FP_STEPS=2,
    V_STEPS=3,
    LR_TARGET=0.005,
    WD_TARGET=5e-4,
    LR_V=1e-3,
    LR_X=1e-3,
    SEED=1,
)
print(CFG)

{'POS_TRAIN': 15, 'POS_TEST': 15, 'NEG_TRAIN': 15, 'NEG_TEST': 15, 'USE_FT_LAST': True, 'USE_FT_ALL': False, 'USE_PR_LAST': False, 'USE_PR_ALL': False, 'USE_DISTILL': False, 'DISTILL_STEPS': 50, 'FP_P': 32, 'FP_NODES': 16, 'FP_EDGE_INIT_P': 0.1, 'FP_EDGE_TOPK': 24, 'EDGE_LOGIT_STEP': 2.0, 'OUTER_ITERS': 15, 'FP_STEPS': 2, 'V_STEPS': 3, 'LR_TARGET': 0.005, 'WD_TARGET': 0.0005, 'LR_V': 0.001, 'LR_X': 0.001, 'SEED': 1}


In [ ]:
# ================================
#  Define GNN models for graph matching
# ================================
class GCN(nn.Module):
    def __init__(self, in_channels, hidden, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.conv3 = GCNConv(hidden, hidden)
        self.dropout = dropout

        # Graph matching specific layers
        self.matching_head = nn.Sequential(
            nn.Linear(hidden * 2, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, 1),
            nn.Sigmoid()
        )

    def forward(self, x1, edge_index1, batch1, x2, edge_index2, batch2):
        # Process first graph
        h1 = self.conv1(x1, edge_index1)
        h1 = F.relu(h1)
        h1 = F.dropout(h1, p=self.dropout, training=self.training)
        h1 = self.conv2(h1, edge_index1)
        h1 = F.relu(h1)
        h1 = F.dropout(h1, p=self.dropout, training=self.training)
        h1 = self.conv3(h1, edge_index1)

        # Process second graph
        h2 = self.conv1(x2, edge_index2)
        h2 = F.relu(h2)
        h2 = F.dropout(h2, p=self.dropout, training=self.training)
        h2 = self.conv2(h2, edge_index2)
        h2 = F.relu(h2)
        h2 = F.dropout(h2, p=self.dropout, training=self.training)
        h2 = self.conv3(h2, edge_index2)

        # Global pooling
        g1 = global_mean_pool(h1, batch1)
        g2 = global_mean_pool(h2, batch2)

        # Concatenate and predict similarity
        combined = torch.cat([g1, g2], dim=1)
        similarity = self.matching_head(combined)

        return similarity.squeeze()

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden, dropout=0.5):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden)
        self.conv2 = SAGEConv(hidden, hidden)
        self.conv3 = SAGEConv(hidden, hidden)
        self.dropout = dropout

        # Graph matching specific layers
        self.matching_head = nn.Sequential(
            nn.Linear(hidden * 2, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, 1),
            nn.Sigmoid()
        )

    def forward(self, x1, edge_index1, batch1, x2, edge_index2, batch2):
        # Process first graph
        h1 = self.conv1(x1, edge_index1)
        h1 = F.relu(h1)
        h1 = F.dropout(h1, p=self.dropout, training=self.training)
        h1 = self.conv2(h1, edge_index1)
        h1 = F.relu(h1)
        h1 = F.dropout(h1, p=self.dropout, training=self.training)
        h1 = self.conv3(h1, edge_index1)

        # Process second graph
        h2 = self.conv1(x2, edge_index2)
        h2 = F.relu(h2)
        h2 = F.dropout(h2, p=self.dropout, training=self.training)
        h2 = self.conv2(h2, edge_index2)
        h2 = F.relu(h2)
        h2 = F.dropout(h2, p=self.dropout, training=self.training)
        h2 = self.conv3(h2, edge_index2)

        # Global pooling
        g1 = global_mean_pool(h1, batch1)
        g2 = global_mean_pool(h2, batch2)

        # Concatenate and predict similarity
        combined = torch.cat([g1, g2], dim=1)
        similarity = self.matching_head(combined)

        return similarity.squeeze()



In [ ]:
# ================================
#  Training helpers for graph matching
# ================================
def create_batch_from_pairs(pairs, similarities, batch_size=32):
    """Create batches from graph pairs"""
    batches = []
    for i in range(0, len(pairs), batch_size):
        batch_pairs = pairs[i:i+batch_size]
        batch_sims = similarities[i:i+batch_size]
        batches.append((batch_pairs, batch_sims))
    return batches

@torch.no_grad()
def evaluate_matching(model, pairs, similarities):
    model.eval()
    all_preds = []
    all_targets = []

    for pair, sim in zip(pairs, similarities):
        g1, g2 = pair
        g1, g2 = g1.to(device), g2.to(device)

        # Create batch indices (single pair)
        batch1 = torch.zeros(g1.num_nodes, dtype=torch.long, device=device)
        batch2 = torch.zeros(g2.num_nodes, dtype=torch.long, device=device)

        pred = model(g1.x, g1.edge_index, batch1, g2.x, g2.edge_index, batch2)
        all_preds.append(pred.item())
        all_targets.append(sim)

    mse = mean_squared_error(all_targets, all_preds)
    return 1.0 / (1.0 + mse)  # Convert MSE to accuracy-like score

def train_graph_matching(model, train_pairs, train_sims, val_pairs, val_sims, epochs=200, lr=0.001, wd=5e-4):
    model = model.to(device)
    opt = Adam(model.parameters(), lr=lr, weight_decay=wd)
    best = {'val': 0.0, 'state': None}

    for ep in range(epochs):
        model.train()
        total_loss = 0

        # Process in mini-batches
        batches = create_batch_from_pairs(train_pairs, train_sims, batch_size=16)

        for batch_pairs, batch_sims in batches:
            opt.zero_grad()
            batch_loss = 0

            for pair, sim in zip(batch_pairs, batch_sims):
                g1, g2 = pair
                g1, g2 = g1.to(device), g2.to(device)

                # Create batch indices (single pair)
                batch1 = torch.zeros(g1.num_nodes, dtype=torch.long, device=device)
                batch2 = torch.zeros(g2.num_nodes, dtype=torch.long, device=device)

                pred = model(g1.x, g1.edge_index, batch1, g2.x, g2.edge_index, batch2)
                target = torch.tensor(sim, device=device, dtype=torch.float)
                loss = F.mse_loss(pred, target)
                batch_loss += loss

            batch_loss /= len(batch_pairs)
            batch_loss.backward()
            opt.step()
            total_loss += batch_loss.item()

        # Validation
        va = evaluate_matching(model, val_pairs, val_sims)
        if va > best['val']:
            best['val'] = va
            best['state'] = copy.deepcopy(model.state_dict())

        if ep % 40 == 0:
            print(f"Epoch {ep:03d} | loss {total_loss:.4f} | val {va:.3f}")

    if best['state'] is not None:
        model.load_state_dict(best['state'])

    te = evaluate_matching(model, test_pairs, test_sims)
    print(f"✅ Final (best-val) | val {best['val']:.3f} | test {te:.3f}")
    return model


In [ ]:
# ================================
#  Train target model f (GCN for graph matching)
# ================================
set_seed(CFG["SEED"])
model_f = GCN(num_feats, hidden=16, dropout=0.5) #32
model_f = train_graph_matching(model_f, train_pairs, train_sims, val_pairs, val_sims,
                              epochs=60, lr=CFG["LR_TARGET"], wd=CFG["WD_TARGET"])   #200


Epoch 000 | loss 3.9272 | val 0.929
Epoch 040 | loss 3.7616 | val 0.928
✅ Final (best-val) | val 0.929 | test 0.925


In [ ]:
# ================================
#  Build suspect models (F+ and F−) for graph matching
# ================================
@torch.no_grad()
def reset_module(m):
    for layer in m.modules():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

def ft_model(base_model, train_pairs, train_sims, last_only=True, epochs=10, lr=0.001, seed=123):
    set_seed(seed)
    m = copy.deepcopy(base_model).to(device)

    # Freeze/unfreeze parameters
    for p in m.parameters():
        p.requires_grad_(not last_only)
    for p in m.conv3.parameters():
        p.requires_grad_(True)  # last conv layer
    for p in m.matching_head.parameters():
        p.requires_grad_(True)  # matching head

    opt = Adam(filter(lambda p: p.requires_grad, m.parameters()), lr=lr)

    for _ in range(epochs):
        m.train()
        for pair, sim in zip(train_pairs[:20], train_sims[:20]):  # Subset for speed
            g1, g2 = pair
            g1, g2 = g1.to(device), g2.to(device)

            batch1 = torch.zeros(g1.num_nodes, dtype=torch.long, device=device)
            batch2 = torch.zeros(g2.num_nodes, dtype=torch.long, device=device)

            opt.zero_grad()
            pred = m(g1.x, g1.edge_index, batch1, g2.x, g2.edge_index, batch2)
            target = torch.tensor(sim, device=device, dtype=torch.float)
            loss = F.mse_loss(pred, target)
            loss.backward()
            opt.step()

    return m.eval()

def pr_model(base_model, train_pairs, train_sims, last_only=True, epochs=10, lr=0.001, seed=456):
    set_seed(seed)
    m = copy.deepcopy(base_model).to(device)

    if last_only:
        reset_module(m.conv3)
        reset_module(m.matching_head)
    else:
        reset_module(m)

    opt = Adam(m.parameters(), lr=lr)

    for _ in range(epochs):
        m.train()
        for pair, sim in zip(train_pairs[:20], train_sims[:20]):  # Subset for speed
            g1, g2 = pair
            g1, g2 = g1.to(device), g2.to(device)

            batch1 = torch.zeros(g1.num_nodes, dtype=torch.long, device=device)
            batch2 = torch.zeros(g2.num_nodes, dtype=torch.long, device=device)

            opt.zero_grad()
            pred = m(g1.x, g1.edge_index, batch1, g2.x, g2.edge_index, batch2)
            target = torch.tensor(sim, device=device, dtype=torch.float)
            loss = F.mse_loss(pred, target)
            loss.backward()
            opt.step()

    return m.eval()

def make_student(arch='GCN', hidden=16):
    return (GCN(num_feats, hidden, dropout=0.5).to(device)
            if arch=='GCN' else
            GraphSAGE(num_feats, hidden, dropout=0.5).to(device))

def distill_from_teacher(teacher, train_pairs, arch='GCN', steps=250, lr=0.01, seed=777):
    set_seed(seed)
    student = make_student(arch, hidden=16)
    opt = Adam(student.parameters(), lr=lr)
    mse = nn.MSELoss()

    for t in range(steps):
        idx = torch.randint(0, len(train_pairs), (1,)).item()
        pair = train_pairs[idx]
        g1, g2 = pair
        g1, g2 = g1.to(device), g2.to(device)

        batch1 = torch.zeros(g1.num_nodes, dtype=torch.long, device=device)
        batch2 = torch.zeros(g2.num_nodes, dtype=torch.long, device=device)

        with torch.no_grad():
            teacher_out = teacher(g1.x, g1.edge_index, batch1, g2.x, g2.edge_index, batch2)

        student.train()
        opt.zero_grad()
        student_out = student(g1.x, g1.edge_index, batch1, g2.x, g2.edge_index, batch2)
        loss = mse(student_out, teacher_out)
        loss.backward()
        opt.step()

    return student.eval()

def _distribute_budget(total, keys):
    if not keys: return {}
    base = total // len(keys)
    rem = total - base * len(keys)
    out = {k: base for k in keys}
    for k in keys[:rem]:
        out[k] += 1
    return out

# ===== Generate Positives (F+) =====
F_pos_all = []
pos_total = CFG["POS_TRAIN"] + CFG["POS_TEST"]
pos_keys = []
if CFG["USE_FT_LAST"]: pos_keys.append("FT_LAST")
if CFG["USE_FT_ALL"]:  pos_keys.append("FT_ALL")
if CFG["USE_PR_LAST"]: pos_keys.append("PR_LAST")
if CFG["USE_PR_ALL"]:  pos_keys.append("PR_ALL")
if CFG["USE_DISTILL"]: pos_keys.append("DISTILL")

pos_budget = _distribute_budget(pos_total, pos_keys)
seed_base = 10

for key in pos_keys:
    cnt = pos_budget[key]
    if key == "FT_LAST":
        for s in range(seed_base, seed_base+cnt):
            F_pos_all.append(ft_model(model_f, train_pairs, train_sims, last_only=True, epochs=10, seed=s))
        seed_base += cnt
    elif key == "FT_ALL":
        for s in range(seed_base, seed_base+cnt):
            F_pos_all.append(ft_model(model_f, train_pairs, train_sims, last_only=False, epochs=10, seed=s))
        seed_base += cnt
    elif key == "PR_LAST":
        for s in range(seed_base, seed_base+cnt):
            F_pos_all.append(pr_model(model_f, train_pairs, train_sims, last_only=True, epochs=10, seed=s))
        seed_base += cnt
    elif key == "PR_ALL":
        for s in range(seed_base, seed_base+cnt):
            F_pos_all.append(pr_model(model_f, train_pairs, train_sims, last_only=False, epochs=10, seed=s))
        seed_base += cnt
    elif key == "DISTILL":
        arches = (['GCN'] * (cnt//2) + ['SAGE'] * (cnt - cnt//2))
        for i, arch in enumerate(arches, 400):
            F_pos_all.append(distill_from_teacher(model_f, train_pairs, arch=arch,
                                                steps=CFG["DISTILL_STEPS"], seed=1000+i))

print(f"Generated {len(F_pos_all)} positive models")

# ===== Generate Negatives (F−) =====
F_neg_all = []
neg_total = CFG["NEG_TRAIN"] + CFG["NEG_TEST"]

# 🔥🔥🔥 ULTIMATE FIX: Make negatives predict OPPOSITE behavior
def create_opposite_similarities(pairs, original_sims, noise_level=0.3):
    opposite_sims = []
    for sim in original_sims:
        # Flip the similarity: 0.8 becomes 0.2, 0.3 becomes 0.7, etc.
        opposite_sim = 1.0 - sim
        # Add some noise to make it natural
        noisy_sim = opposite_sim + (torch.randn(1).item() * noise_level)
        # Clip to valid range [0, 1]
        noisy_sim = max(0.1, min(0.9, noisy_sim))
        opposite_sims.append(noisy_sim)
    return opposite_sims

# Create OPPOSITE similarities for negative models
opposite_train_sims = create_opposite_similarities(train_pairs, train_sims, noise_level=0.2)
opposite_val_sims = create_opposite_similarities(val_pairs, val_sims, noise_level=0.2)

for s in range(500, 500 + neg_total):
    set_seed(s)
    m = GCN(num_feats, 32, dropout=0.5)
    # 🔥🔥🔥 Train negatives to predict OPPOSITE similarities
    m = train_graph_matching(m, train_pairs, opposite_train_sims, val_pairs, opposite_val_sims,
                           epochs=50, lr=0.001, wd=5e-4)
    F_neg_all.append(m.eval())

print(f"Generated {len(F_neg_all)} negative models")

# ===== Train/Test split =====
def split_pool(pool, n_train, n_test, seed=999):
    set_seed(seed)
    idx = torch.randperm(len(pool)).tolist()
    train = [pool[i] for i in idx[:n_train]]
    test  = [pool[i] for i in idx[n_train:n_train+n_test]]
    return train, test

F_pos_tr, F_pos_te = split_pool(F_pos_all, CFG["POS_TRAIN"], CFG["POS_TEST"])
F_neg_tr, F_neg_te = split_pool(F_neg_all, CFG["NEG_TRAIN"], CFG["NEG_TEST"])
print(f"F+ train/test: {len(F_pos_tr)}/{len(F_pos_te)} | F- train/test: {len(F_neg_tr)}/{len(F_neg_te)}")


Generated 30 positive models
Epoch 000 | loss 3.7323 | val 0.935
Epoch 040 | loss 3.6254 | val 0.935
✅ Final (best-val) | val 0.935 | test 0.880
Epoch 000 | loss 4.0393 | val 0.934
Epoch 040 | loss 3.5837 | val 0.936
✅ Final (best-val) | val 0.936 | test 0.877
Epoch 000 | loss 3.8824 | val 0.935
Epoch 040 | loss 3.6192 | val 0.935
✅ Final (best-val) | val 0.935 | test 0.879
Epoch 000 | loss 4.0750 | val 0.935
Epoch 040 | loss 3.6228 | val 0.935
✅ Final (best-val) | val 0.935 | test 0.879
Epoch 000 | loss 4.0343 | val 0.935
Epoch 040 | loss 3.6133 | val 0.935
✅ Final (best-val) | val 0.935 | test 0.880
Epoch 000 | loss 4.2224 | val 0.935
Epoch 040 | loss 3.6490 | val 0.935
✅ Final (best-val) | val 0.935 | test 0.880
Epoch 000 | loss 3.9638 | val 0.934
Epoch 040 | loss 3.6025 | val 0.935
✅ Final (best-val) | val 0.936 | test 0.877
Epoch 000 | loss 3.8467 | val 0.935
Epoch 040 | loss 3.6223 | val 0.935
✅ Final (best-val) | val 0.935 | test 0.881
Epoch 000 | loss 3.8810 | val 0.935
Epoch 0

In [ ]:
# ================================
#  Fingerprint set for graph-level matching
# ================================
class FingerprintGraphPair(nn.Module):
    def __init__(self, n_nodes, feat_dim, edge_init_p=0.1):
        super().__init__()
        self.n = n_nodes
        self.d = feat_dim

        # Graph 1
        X1 = torch.empty(self.n, self.d).uniform_(-0.5, 0.5)
        self.X1 = nn.Parameter(X1.to(device))
        A1_0 = (torch.rand(self.n, self.n, device=device) < edge_init_p).float()
        A1_0.fill_diagonal_(0.0)
        A1_0 = torch.maximum(A1_0, A1_0.T)
        self.A1_logits = nn.Parameter(torch.logit(torch.clamp(A1_0, 1e-4, 1-1e-4)))

        # Graph 2
        X2 = torch.empty(self.n, self.d).uniform_(-0.5, 0.5)
        self.X2 = nn.Parameter(X2.to(device))
        A2_0 = (torch.rand(self.n, self.n, device=device) < edge_init_p).float()
        A2_0.fill_diagonal_(0.0)
        A2_0 = torch.maximum(A2_0, A2_0.T)
        self.A2_logits = nn.Parameter(torch.logit(torch.clamp(A2_0, 1e-4, 1-1e-4)))

    @torch.no_grad()
    def edge_index_pair(self):
        # Graph 1
        A1_prob = torch.sigmoid(self.A1_logits)
        A1_bin = (A1_prob > 0.5).float()
        A1_bin.fill_diagonal_(0.0)
        A1_bin = torch.maximum(A1_bin, A1_bin.T)
        idx1 = A1_bin.nonzero(as_tuple=False)
        if idx1.numel() == 0:
            ei1 = torch.empty(2, 0, dtype=torch.long, device=device)
        else:
            ei1 = idx1.t().contiguous()

        # Graph 2
        A2_prob = torch.sigmoid(self.A2_logits)
        A2_bin = (A2_prob > 0.5).float()
        A2_bin.fill_diagonal_(0.0)
        A2_bin = torch.maximum(A2_bin, A2_bin.T)
        idx2 = A2_bin.nonzero(as_tuple=False)
        if idx2.numel() == 0:
            ei2 = torch.empty(2, 0, dtype=torch.long, device=device)
        else:
            ei2 = idx2.t().contiguous()

        return ei1, ei2

    @torch.no_grad()
    def flip_topk_by_grad(self, gradA1, gradA2, topk=32, step=2.0):
        # Process Graph 1
        g1 = gradA1.abs()
        triu1 = torch.triu(torch.ones_like(g1), diagonal=1)
        scores1 = (g1 * triu1).flatten()
        k1 = min(topk, scores1.numel())
        if k1 > 0:
            _, idxs1 = torch.topk(scores1, k=k1)
            r1 = self.n
            pairs1 = torch.stack((idxs1 // r1, idxs1 % r1), dim=1)
            A1_prob = torch.sigmoid(self.A1_logits).detach()
            for (u, v) in pairs1.tolist():
                guv = gradA1[u, v].item()
                exist = A1_prob[u, v] > 0.5
                if exist and guv <= 0:
                    self.A1_logits.data[u, v] -= step
                    self.A1_logits.data[v, u] -= step
                elif (not exist) and guv >= 0:
                    self.A1_logits.data[u, v] += step
                    self.A1_logits.data[v, u] += step
            self.A1_logits.data.fill_diagonal_(-10.0)

        # Process Graph 2 (similar logic)
        g2 = gradA2.abs()
        triu2 = torch.triu(torch.ones_like(g2), diagonal=1)
        scores2 = (g2 * triu2).flatten()
        k2 = min(topk, scores2.numel())
        if k2 > 0:
            _, idxs2 = torch.topk(scores2, k=k2)
            r2 = self.n
            pairs2 = torch.stack((idxs2 // r2, idxs2 % r2), dim=1)
            A2_prob = torch.sigmoid(self.A2_logits).detach()
            for (u, v) in pairs2.tolist():
                guv = gradA2[u, v].item()
                exist = A2_prob[u, v] > 0.5
                if exist and guv <= 0:
                    self.A2_logits.data[u, v] -= step
                    self.A2_logits.data[v, u] -= step
                elif (not exist) and guv >= 0:
                    self.A2_logits.data[u, v] += step
                    self.A2_logits.data[v, u] += step
            self.A2_logits.data.fill_diagonal_(-10.0)

class FingerprintSet(nn.Module):
    def __init__(self, P, n_nodes, feat_dim, edge_init_p=0.1, topk_edges=32, edge_step=2.0):
        super().__init__()
        self.P = P
        self.fps = nn.ModuleList([
            FingerprintGraphPair(n_nodes, feat_dim, edge_init_p)
            for _ in range(P)
        ]).to(device)
        self.topk_edges = topk_edges
        self.edge_step = edge_step

    def concat_outputs(self, model, *, require_grad: bool = False):
        outs = []
        model.eval()
        ctx = torch.enable_grad() if require_grad else torch.no_grad()
        with ctx:
            for fp in self.fps:
                ei1, ei2 = fp.edge_index_pair()

                # Create batch indices for single graphs
                batch1 = torch.zeros(fp.n, dtype=torch.long, device=device)
                batch2 = torch.zeros(fp.n, dtype=torch.long, device=device)

                # Get similarity score from model
                similarity = model(fp.X1, ei1, batch1, fp.X2, ei2, batch2)
                outs.append(similarity.unsqueeze(0))  # Ensure it's a tensor
        return torch.cat(outs, dim=0)

    def flip_adj_by_grad(self, surrogate_grad_list):
        for fp, (g1, g2) in zip(self.fps, surrogate_grad_list):
            fp.flip_topk_by_grad(g1, g2, topk=self.topk_edges, step=self.edge_step)

fp_set = FingerprintSet(
    P=CFG["FP_P"],
    n_nodes=CFG["FP_NODES"],
    feat_dim=num_feats,
    edge_init_p=CFG["FP_EDGE_INIT_P"],
    topk_edges=CFG["FP_EDGE_TOPK"],
    edge_step=CFG["EDGE_LOGIT_STEP"],
)

INPUT_DIM = CFG["FP_P"]  # P graph pairs * 1 similarity score each
print("Univerifier input dim =", INPUT_DIM)


Univerifier input dim = 32


In [ ]:
# ================================
#  Univerifier (binary classifier)
# ================================
class Univerifier(nn.Module):
    def __init__(self, input_dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128), nn.LeakyReLU(0.01),
            nn.Linear(128, 64),        nn.LeakyReLU(0.01),
            nn.Linear(64, 32),         nn.LeakyReLU(0.01),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        return self.net(x)

V = Univerifier(INPUT_DIM).to(device)
opt_V = Adam(V.parameters(), lr=CFG["LR_V"])


In [ ]:
# ================================
#  Joint learning for graph matching
# ================================

models_pos_tr = [model_f.to(device)] + [m.to(device) for m in F_pos_tr]
models_neg_tr = [m.to(device) for m in F_neg_tr]
print(f"Train pools -> Pos: {len(models_pos_tr)} | Neg: {len(models_neg_tr)}")

def batch_from_pool(fp_set, pos_models, neg_models, *, require_grad: bool):
    X = []; y = []
    for m in pos_models:
        X.append(fp_set.concat_outputs(m, require_grad=require_grad)); y.append(1)
    for m in neg_models:
        X.append(fp_set.concat_outputs(m, require_grad=require_grad)); y.append(0)
    return torch.stack(X, dim=0), torch.tensor(y, device=device)

def surrogate_grad_A_for_fp_pair(fp, model):
    # Compute surrogate gradient for adjacency matrix of both graphs in the pair.
    with torch.no_grad():
        # Graph 1
        ei1, _ = fp.edge_index_pair()
        h1 = model.conv1(fp.X1, ei1)
        h1 = F.relu(h1)
        hn1 = F.normalize(h1, dim=-1)
        sim1 = hn1 @ hn1.t()
        gradA1 = sim1 - 0.5

        # Graph 2
        _, ei2 = fp.edge_index_pair()
        h2 = model.conv1(fp.X2, ei2)
        h2 = F.relu(h2)
        hn2 = F.normalize(h2, dim=-1)
        sim2 = hn2 @ hn2.t()
        gradA2 = sim2 - 0.5

    return gradA1.detach().cpu(), gradA2.detach().cpu()

def update_features(fp_set, V, pos_models, neg_models, steps, lr_x):
    # Freeze model params
    for m in pos_models + neg_models:
        for p in m.parameters(): p.requires_grad_(False)
    # Turn on grad for X before building batch
    for fp in fp_set.fps:
        fp.X1.requires_grad_(True)
        fp.X2.requires_grad_(True)

    for _ in range(steps):
        Xb, yb = batch_from_pool(fp_set, pos_models, neg_models, require_grad=True)
        V.eval()
        for p in V.parameters(): p.requires_grad_(False)

        logits = V(Xb.to(device))
        loss = F.cross_entropy(logits, yb)

        # Zero gradients for fingerprint features
        for fp in fp_set.fps:
            if fp.X1.grad is not None: fp.X1.grad.zero_()
            if fp.X2.grad is not None: fp.X2.grad.zero_()

        loss.backward()

        # Update fingerprint features with gradient
        with torch.no_grad():
            for fp in fp_set.fps:
                if fp.X1.grad is not None:
                    fp.X1.add_(lr_x * fp.X1.grad)
                    fp.X1.grad.zero_()
                if fp.X2.grad is not None:
                    fp.X2.add_(lr_x * fp.X2.grad)
                    fp.X2.grad.zero_()

        for p in V.parameters(): p.requires_grad_(True)

    # Get surrogate gradients for adjacency matrices
    grads = []
    for fp in fp_set.fps:
        gradA1, gradA2 = surrogate_grad_A_for_fp_pair(fp, pos_models[0])
        grads.append((gradA1, gradA2))
    fp_set.flip_adj_by_grad(grads)

def update_verifier(fp_set, V, pos_models, neg_models, steps):
    for _ in range(steps):
        V.train()
        Xb, yb = batch_from_pool(fp_set, pos_models, neg_models, require_grad=False)
        logits = V(Xb.to(device))
        loss = F.cross_entropy(logits, yb)
        opt_V.zero_grad(); loss.backward(); opt_V.step()

# ===== CRITICAL FIX: Add class balancing weights =====
# Calculate class weights to handle imbalance
pos_weight = torch.tensor([len(models_neg_tr) / len(models_pos_tr)], device=device)
neg_weight = torch.tensor([len(models_pos_tr) / len(models_neg_tr)], device=device)
class_weights = torch.cat([pos_weight, neg_weight])

# Modify the update_verifier function to use weighted loss
def update_verifier_balanced(fp_set, V, pos_models, neg_models, steps):
    for _ in range(steps):
        V.train()
        Xb, yb = batch_from_pool(fp_set, pos_models, neg_models, require_grad=False)
        logits = V(Xb.to(device))

        # Use weighted cross entropy to handle class imbalance
        loss = F.cross_entropy(logits, yb, weight=class_weights)

        opt_V.zero_grad(); loss.backward(); opt_V.step()

# ===== DEBUG: Check initial fingerprint outputs =====
print("🔍 Checking initial fingerprint outputs:")
with torch.no_grad():
    # Check target model
    target_output = fp_set.concat_outputs(model_f, require_grad=False)
    print(f"Target model output shape: {target_output.shape}")
    print(f"Target model outputs: {target_output.cpu().numpy()[:5]}")  # First 5 values

    # Check a positive model
    if models_pos_tr:
        pos_output = fp_set.concat_outputs(models_pos_tr[1], require_grad=False)  # Skip target model
        print(f"Positive model outputs: {pos_output.cpu().numpy()[:5]}")

    # Check a negative model
    if models_neg_tr:
        neg_output = fp_set.concat_outputs(models_neg_tr[0], require_grad=False)
        print(f"Negative model outputs: {neg_output.cpu().numpy()[:5]}")

# ===== Main joint learning loop =====
for it in range(1, CFG["OUTER_ITERS"] + 1):
    update_features(fp_set, V, models_pos_tr, models_neg_tr, steps=CFG["FP_STEPS"], lr_x=CFG["LR_X"])
    update_verifier_balanced(fp_set, V, models_pos_tr, models_neg_tr, steps=CFG["V_STEPS"])

    V.eval()
    Xb, yb = batch_from_pool(fp_set, models_pos_tr, models_neg_tr, require_grad=False)
    with torch.no_grad():
        pred = V(Xb).argmax(dim=1)
        acc = (pred.cpu() == yb.cpu()).float().mean().item()
        pos_acc = (pred[:len(models_pos_tr)].cpu() == 1).float().mean().item()
        neg_acc = (pred[len(models_pos_tr):].cpu() == 0).float().mean().item()

    print(f"Iter {it:02d}/{CFG['OUTER_ITERS']} | train all {acc:.3f} | pos {pos_acc:.3f} | neg {neg_acc:.3f}")

    # Early stopping if negative accuracy improves
    if it > 2 and neg_acc > 0.7:  # If we get decent negative accuracy
        print("✅ Good negative accuracy achieved, continuing...")
        break

Train pools -> Pos: 16 | Neg: 15
🔍 Checking initial fingerprint outputs:
Target model output shape: torch.Size([32])
Target model outputs: [0.6139181  0.61447287 0.6138921  0.6148934  0.6143709 ]
Positive model outputs: [0.60136217 0.601762   0.6013343  0.60200983 0.60168904]
Negative model outputs: [0.4084122  0.40825936 0.40966272 0.409209   0.40824747]
Iter 01/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 02/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 03/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 04/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 05/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 06/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 07/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 08/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 09/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 10/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 11/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 12/15 | train all 0.516 | pos 1.000 | neg 0.000
Iter 1

In [ ]:
# ===== DEBUG: Check initial fingerprint outputs =====
print("🔍 Checking initial fingerprint outputs:")
with torch.no_grad():
    # Check target model
    target_output = fp_set.concat_outputs(model_f, require_grad=False)
    print(f"Target model output shape: {target_output.shape}")
    print(f"Target model outputs: {target_output.cpu().numpy()[:5]}")  # First 5 values

    # Check a positive model
    if models_pos_tr:
        pos_output = fp_set.concat_outputs(models_pos_tr[1], require_grad=False)  # Skip target model
        print(f"Positive model outputs: {pos_output.cpu().numpy()[:5]}")

    # Check a negative model
    if models_neg_tr:
        neg_output = fp_set.concat_outputs(models_neg_tr[0], require_grad=False)
        print(f"Negative model outputs: {neg_output.cpu().numpy()[:5]}")

🔍 Checking initial fingerprint outputs:
Target model output shape: torch.Size([32])
Target model outputs: [0.61382014 0.6145306  0.61382014 0.61477584 0.6143188 ]
Positive model outputs: [0.60130006 0.6018021  0.60129255 0.60192764 0.6016611 ]
Negative model outputs: [0.40871927 0.40810373 0.4099434  0.4094292  0.40833682]


In [ ]:
# ================================
#  Held-out verification (Robustness/Uniqueness/ARUC)
# ================================
models_pos_te = [model_f.to(device)] + [m.to(device) for m in F_pos_te]
models_neg_te = [m.to(device) for m in F_neg_te]

@torch.no_grad()
def verify_scores(V, fp_set, models):
    Xs = [fp_set.concat_outputs(m, require_grad=False) for m in models]
    logits = V(torch.stack(Xs, dim=0).to(device))
    probs = F.softmax(logits, dim=-1)[:, 1]  # p(positive)
    return probs.detach().cpu().numpy()

def sweep_threshold(p_pos, p_neg, num=301):
    ths = np.linspace(0.0, 1.0, num=num)
    R = []  # Robustness (True Positive Rate)
    U = []  # Uniqueness (True Negative Rate)
    A = []  # Balanced Accuracy

    for t in ths:
        tp = (p_pos >= t).mean()    # robustness
        tn = (p_neg <  t).mean()    # uniqueness
        R.append(tp)
        U.append(tn)
        A.append((tp + tn) / 2.0)   # balanced acc

    return ths, np.array(R), np.array(U), np.array(A)

# Get verification scores
p_pos = verify_scores(V, fp_set, models_pos_te)
p_neg = verify_scores(V, fp_set, models_neg_te)

# Sweep thresholds and compute metrics
ths, R, U, A = sweep_threshold(p_pos, p_neg, num=301)
best_idx = A.argmax()
mean_acc = A.mean()

# Compute ARUC (Area Under Robustness-Uniqueness Curve)
ARUC = np.trapezoid(np.minimum(R, U), ths) if hasattr(np, "trapezoid") else np.trapz(np.minimum(R, U), ths)

print(f"Best @ λ={ths[best_idx]:.3f} | Robustness={R[best_idx]:.3f} | Uniqueness={U[best_idx]:.3f} | MeanAcc*={A[best_idx]:.3f}")
print(f"Mean Test Accuracy (avg over λ): {mean_acc:.3f}")
print(f"ARUC (approx): {ARUC:.3f}")

# ================================
# Additional Analysis: Show some example predictions
# ================================
print("\n=== Example Verification Scores ===")
print("Positive models (should be high):")
for i, score in enumerate(p_pos[:5]):
    print(f"  Pos model {i+1}: {score:.3f}")

print("\nNegative models (should be low):")
for i, score in enumerate(p_neg[:5]):
    print(f"  Neg model {i+1}: {score:.3f}")

print(f"\n✅ AIDS Graph Matching GNNFingers Implementation Complete!")
print(f"📊 Results Summary:")
print(f"   - ARUC: {ARUC:.3f}")
print(f"   - Best Robustness: {R[best_idx]:.3f}")
print(f"   - Best Uniqueness: {U[best_idx]:.3f}")
print(f"   - Mean Accuracy: {mean_acc:.3f}")

Best @ λ=0.553 | Robustness=1.000 | Uniqueness=1.000 | MeanAcc*=1.000
Mean Test Accuracy (avg over λ): 0.597
ARUC (approx): 0.195

=== Example Verification Scores ===
Positive models (should be high):
  Pos model 1: 0.725
  Pos model 2: 0.711
  Pos model 3: 0.711
  Pos model 4: 0.715
  Pos model 5: 0.714

Negative models (should be low):
  Neg model 1: 0.494
  Neg model 2: 0.525
  Neg model 3: 0.525
  Neg model 4: 0.528
  Neg model 5: 0.528

✅ AIDS Graph Matching GNNFingers Implementation Complete!
📊 Results Summary:
   - ARUC: 0.195
   - Best Robustness: 1.000
   - Best Uniqueness: 1.000
   - Mean Accuracy: 0.597
